In [1]:
library(cli)
library(karyoploteR)
library(tidyverse)
library(GenomicRanges)
library(regioneR)
library(readxl)
library(dplyr)

Loading required package: regioneR

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading

In [50]:
cen_table<- read_csv('../../data/CEN/cen_info.csv')%>%as.data.frame%>%
  mutate(
    start = as.numeric(start),
    end = as.numeric(end),
    GC = as.numeric(GC),
    CpG = as.numeric(CpG),
    TE_cov = as.numeric(TE_cov),
    mCpG = as.numeric(mCpG),
    `mCpG/CpG` = as.numeric(`mCpG/CpG`)  
  )

Rows: 20030 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): chrom
dbl (8): start, end, GC, CpG, TE_cov, mCpG, gene_cov, mCpG/CpG

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [51]:
to_minus<-cen_table%>%group_by(chrom)%>%summarise(to_minus = min(start))%>%as.data.frame

In [52]:
#left join to_minus to cen_table by chrom
new_cen_table <-left_join(cen_table, to_minus, by ='chrom')

In [ ]:
#new_cen_table <-new_cen_table%>%mutate(start = start - to_minus, end = end- to_minus)

In [53]:
genome_size.c1<-cen_table%>%select(chrom)%>%unique
genome_size.c2<-cen_table%>%group_by(chrom)%>%summarise(min(start))%>%as.data.frame%>%select('min(start)')
genome_size.c3<-cen_table%>%group_by(chrom)%>%summarise(max(end))%>%as.data.frame%>%select('max(end)')
genome<-data.frame(chr=genome_size.c1$chrom, start=genome_size.c2$`min(start)`, end=genome_size.c3$`max(end)`)

In [54]:
cen_position <-read.table('../../data/20250220_cen.table', header = F, sep = '\t')
colnames(cen_position)<-c('chrom', 'start','end')

In [55]:
new_cen_position <-left_join(cen_position, to_minus, by = 'chrom')
new_cen_position <-new_cen_position%>%mutate(start = start - to_minus, end = end- to_minus)

In [56]:
partA<-genome%>%head(18)%>%toGRanges
partB<-genome%>%tail(18)%>%toGRanges

In [57]:
GC_data<-cen_table%>%select(chrom, start,end, GC)%>%as.data.frame%>%toGRanges
GC_data


GRanges object with 20030 ranges and 1 metadata column:
        seqnames            ranges strand |        GC
           <Rle>         <IRanges>  <Rle> | <numeric>
      1   Chr01A 33700000-33710000      * |     0.337
      2   Chr01A 33710000-33720000      * |     0.379
      3   Chr01A 33720000-33730000      * |     0.368
      4   Chr01A 33730000-33740000      * |     0.433
      5   Chr01A 33740000-33750000      * |     0.388
    ...      ...               ...    ... .       ...
  20026   Chr18B   7290000-7300000      * |     0.332
  20027   Chr18B   7300000-7310000      * |     0.329
  20028   Chr18B   7310000-7320000      * |     0.305
  20029   Chr18B   7320000-7330000      * |     0.299
  20030   Chr18B   7330000-7340000      * |     0.378
  -------
  seqinfo: 36 sequences from an unspecified genome; no seqlengths

In [61]:
options(repr.plot.width=12, repr.plot.height=16)
pdf("../../data/20250221_Au3_partA_cen.pdf", width=12, height=16)
kp <- plotKaryotype(genome = partA, plot.type=2)
kpAddBaseNumbers(kp, tick.dist=1000000, tick.len=10, add.units=TRUE, digits=2, minor.ticks=TRUE, minor.tick.dist=100000, minor.tick.len=5,  cex=0.7, tick.col='black', minor.tick.col='red')
kpRect(kp, chr=cen_position$chrom, x0 = cen_position$start, x1 = cen_position$end, y0 = -2, y1 = 0.5, col = rgb(0, 0, 1, 0.2), border = "NA")
#kpHeatmap(kp, GC_data, y = cen_table$TE_cov, col=c("white","yellow","red"),ymin=0, ymax=1,lwd=0.2, data.panel = "ideogram" )
#kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$GC), r0=0, r1=0.5,col="darkgreen",ymin=0, ymax=1,lwd=0, data.panel = 1 )
#kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$CpG), r0=0.8, r1=0.2,col="red",ymin=0, ymax=100,lwd=0.3, data.panel = 2 )
#kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$mCpG), r0=0.8, r1=0.2,col="blue",ymin=0, ymax=100,lwd=0.3, data.panel = 2 )
kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$GC),r0=0, r1=0.5,col="orange",ymin=0, ymax=1,lwd=0.5, data.panel = 1 )
kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$TE_cov), r0=0.8, r1=0.5,col="blue",ymin=0, ymax=1,lwd=0.5, data.panel = 2 )
kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$gene_cov), r0=0.5, r1=0.2,col="red",ymin=0, ymax=1,lwd=0.5, data.panel = 2 )
kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$`mCpG/CpG`), r0=1.5, r1=0.9,col="black",ymin=0, ymax=1,lwd=0.5, data.panel = 2 )
kpAxis(kp, ymax=100, ymin=0, r0=0, r1=0.5, cex=0.5)
kpAxis(kp, ymax=100, ymin=0, r0=0.8, r1=0.2, cex=0.5, data.panel = 2)
kpAxis(kp, ymax=100, ymin=0, r0=1.5, r1=0.9, cex=0.5, data.panel = 2)
kpDataBackground(kp, data.panel = 2, col=rgb(0.219, 0.216, 0.206, 0.1),r0 =0.8, r1=0.2)
kpDataBackground(kp, data.panel = 1, col=rgb(0.219, 0.216, 0.206, 0.1),r0 =0, r1=0.5)
kpDataBackground(kp, data.panel = 2, col=rgb(0.219, 0.216, 0.206, 0.1),r0=1.5, r1=0.9)
dev.off()


No predefined canonical chromosomes found for the requested genome. Applying a heuristic chromosome filtering.

To get the unfiltered genome, please set chromosomes="all" in the plotKaryotype call



png 
  2

In [62]:
options(repr.plot.width=12, repr.plot.height=16)
pdf("../../data/20250221_Au3_partB_cen.pdf", width=12, height=16)
kp <- plotKaryotype(genome = partB, plot.type=2)
kpAddBaseNumbers(kp, tick.dist=1000000, tick.len=10, add.units=TRUE, digits=2, minor.ticks=TRUE, minor.tick.dist=100000, minor.tick.len=5,  cex=0.7, tick.col='black', minor.tick.col='red')
kpRect(kp, chr=cen_position$chrom, x0 = cen_position$start, x1 = cen_position$end, y0 = -2, y1 = 0.5, col = rgb(0, 0, 1, 0.2), border = "NA")
#kpHeatmap(kp, GC_data, y = cen_table$TE_cov, col=c("white","yellow","red"),ymin=0, ymax=1,lwd=0.2, data.panel = "ideogram" )
#kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$GC), r0=0, r1=0.5,col="darkgreen",ymin=0, ymax=1,lwd=0, data.panel = 1 )
#kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$CpG), r0=0.8, r1=0.2,col="red",ymin=0, ymax=100,lwd=0.3, data.panel = 2 )
#kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$mCpG), r0=0.8, r1=0.2,col="blue",ymin=0, ymax=100,lwd=0.3, data.panel = 2 )
kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$GC),r0=0, r1=0.5,col="orange",ymin=0, ymax=1,lwd=0.3, data.panel = 1 )
kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$TE_cov), r0=0.8, r1=0.5,col="blue",ymin=0, ymax=1,lwd=0.3, data.panel = 2 )
kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$gene_cov), r0=0.5, r1=0.2,col="red",ymin=0, ymax=1,lwd=0.3, data.panel = 2 )
kpLines(kp, chr =cen_table$chrom, x=(cen_table$start +cen_table$end)/2, y = (cen_table$`mCpG/CpG`), r0=1.5, r1=0.9,col="black",ymin=0, ymax=1,lwd=0.2, data.panel = 2 )
kpAxis(kp, ymax=100, ymin=0, r0=0, r1=0.5, cex=0.5)
kpAxis(kp, ymax=100, ymin=0, r0=0.8, r1=0.2, cex=0.5, data.panel = 2)
kpAxis(kp, ymax=100, ymin=0, r0=1.5, r1=0.9, cex=0.5, data.panel = 2)
kpDataBackground(kp, data.panel = 2, col=rgb(0.219, 0.216, 0.206, 0.1),r0 =0.8, r1=0.2)
kpDataBackground(kp, data.panel = 1, col=rgb(0.219, 0.216, 0.206, 0.1),r0 =0, r1=0.5)
kpDataBackground(kp, data.panel = 2, col=rgb(0.219, 0.216, 0.206, 0.1),r0=1.5, r1=0.9)
dev.off()


No predefined canonical chromosomes found for the requested genome. Applying a heuristic chromosome filtering.

To get the unfiltered genome, please set chromosomes="all" in the plotKaryotype call



png 
  2